In [12]:
import modules.Extract.entrez as en
import xml.dom.minidom as minidom

In [13]:
response = en.esearch(database="nucleotide",term="Chikungunya[title]",retmax=100,datetype="pdat",reldate=60) # Retorna UID list que tem data de publicação

In [14]:
decoded_response = response.content.decode('utf-8')

In [21]:
dom_tree = minidom.parseString(decoded_response)

In [22]:
root = dom_tree.documentElement

In [27]:
list_of_ids = []
for element in root.getElementsByTagName("Id"):
    list_of_ids.append(element.firstChild.data)